In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import parallel_backend
import xgboost as xgb

In [2]:
data = pd.read_csv('fashion-mnist_train.csv')
test_data = pd.read_csv('fashion-mnist_test.csv')
data = data.sample(frac=0.2, replace=True, random_state=1)
test_data = test_data.sample(frac=0.2, replace=True, random_state=1)

In [3]:
features_train = data.drop('label',axis=1)
target_train = data.label
features_test = test_data.drop('label',axis=1)
target_test = test_data.label

In [4]:
features_train.shape,target_train.shape

((12000, 784), (12000,))

In [5]:
#Use StandardScaler to scale pixel values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler_fit = scaler.fit(features_train)
#transform the dataset
X_train = scaler_fit.transform(features_train)
X_test = scaler_fit.transform(features_test)

In [6]:
xgb_clf = xgb.XGBClassifier(tree_method="gpu_hist", predicator="gpu_predicator", verbosity=1,
                           eval_metric=["merror", "map","auc"], objective="multi:softmax",n_jobs=-1)
xgb_model = xgb_clf.fit(X_train, target_train)

In [7]:
#predictions
y_train_preds = xgb_model.predict(X_train)
y_test_preds = xgb_model.predict(X_test)

In [8]:
#Print F1 scores and accuracy
print("Training F1 Micro Average : ", f1_score(target_train, y_train_preds, average="micro"))
print("Test F1 Micro Average : ", f1_score(target_test, y_test_preds, average="micro"))
print("Test Accuracy : ", accuracy_score(target_test, y_test_preds))

Training F1 Micro Average :  0.9201666666666667
Test F1 Micro Average :  0.8575
Test Accuracy :  0.8575


In [6]:
#Gridsearchcv - GPU
from sklearn.model_selection import RandomizedSearchCV

xgb_clf = xgb.XGBClassifier(tree_method="gpu_hist", predicator="gpu_predicator", verbosity=1,
                           eval_metric=["merror", "map","auc"], objective="multi:softmax")

#create parameter grid

parameters = {"learning_rate": [0.1, 0.01, 0.001],
               "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
               "max_depth": [2, 4, 7, 10],
               "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
               "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
               "reg_alpha": [0, 0.5, 1],
               "reg_lambda": [1, 1.5, 2, 3, 4.5],
               "min_child_weight": [1, 3, 5, 7],
               "n_estimators": [100, 250, 500, 1000]}

#create the RandomizedSearchCV
xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring="f1_micro",
                             cv=3, verbose=3, random_state=40)

#increase the CV to 7 when training on a high powered gpu else it will train for 2-3 hours

#fit the model
model_xgboost = xgb_rscv.fit(X_train, target_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.831, total=  20.5s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   20.4s remaining:    0.0s


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.834, total=  19.6s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   40.0s remaining:    0.0s


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=0.834, total=  19.7s
[CV] subsample=0.4, reg_lambda=4.5, reg_alpha=1, n_estimators=100, min_child_weight=1, max_depth=2, learning_rate=0.01, gamma=2, colsample_bytree=0.8 
[CV]  subsample=0.4, reg_lambda=4.5, reg_alpha=1, n_estimators=100, min_child_weight=1, max_depth=2, learning_rate=0.01, gamma=2, colsample_bytree=0.8, score=0.744, total=  19.1s
[CV] subsample=0.4, reg_lambda=4.5, reg_alpha=1, n_estimators=100, min_child_weight=1, max_depth=2, learning_rate=0.01, gamma=2, colsample_bytree=0.8 
[CV]  subsample=0.4, reg_lambda=4.5, reg_alpha=1, n_estimators=100, min_child_weight=1, max_depth=2, learning_rate=0.01, gamma=2, colsample_bytree=0.8, score=0.750, total=  19.3s
[CV] subsample=0.4, reg_lambda=4.5, reg_alpha=1, n_estimators=100, min_child_weight=1, max_depth=2, learning_rate=0.01, gamma=2, colsample_bytree=0.8 
[CV]  subsa

[CV]  subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6, score=0.824, total= 1.9min
[CV] subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6 
[CV]  subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6, score=0.826, total= 1.9min
[CV] subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6 
[CV]  subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6, score=0.823, total= 1.9min


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 79.6min finished


In [7]:
model_xgboost.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6,
              eval_metric=['merror', 'map', 'auc'], gamma=0.3,
              learning_rate=0.01, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='multi:softprob',
              predicator='gpu_predicator', random_state=0, reg_alpha=0,
              reg_lambda=1.5, scale_pos_weight=1, seed=None, silent=None,
              subsample=0.4, tree_method='gpu_hist', verbosity=1)

In [8]:
model_xgboost.best_params_

{'subsample': 0.4,
 'reg_lambda': 1.5,
 'reg_alpha': 0,
 'n_estimators': 1000,
 'min_child_weight': 1,
 'max_depth': 7,
 'learning_rate': 0.01,
 'gamma': 0.3,
 'colsample_bytree': 0.6}

In [9]:
model_xgboost.best_score_

0.8815

In [6]:
xgb_clf = xgb.XGBClassifier(tree_method="gpu_hist", predicator="gpu_predicator", verbosity=1,
                           eval_metric=["merror", "map","auc"], objective="multi:softmax",n_jobs=-1,
                            subsample= 0.4,reg_lambda=1.5,reg_alpha= 0,n_estimators= 1000,min_child_weight= 1,
                            max_depth= 7,learning_rate= 0.01,gamma= 0.3,colsample_bytree= 0.6)
xgb_model = xgb_clf.fit(X_train, target_train)
#predictions
y_train_preds = xgb_model.predict(X_train)
y_test_preds = xgb_model.predict(X_test)

In [7]:
#Print F1 scores and accuracy
print("Training F1 Micro Average : ", f1_score(target_train, y_train_preds, average="micro"))
print("Test F1 Micro Average : ", f1_score(target_test, y_test_preds, average="micro"))
print("Test Accuracy : ", accuracy_score(target_test, y_test_preds))

Training F1 Micro Average :  0.9964166666666666
Test F1 Micro Average :  0.8855
Test Accuracy :  0.8855


In [ ]:
#Gridsearchcv - CPU
xgb_clf = xgb.XGBClassifier(tree_method = "hist", predictor = "cpu_predictor", verbosity = True,
                            objective = "multi:softmax")

# Create parameter grid
parameters = {"learning_rate": [0.1, 0.01, 0.001],
               "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
               "max_depth": [2, 4, 7, 10],
               "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
               "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
               "reg_alpha": [0, 0.5, 1],
               "reg_lambda": [1, 1.5, 2, 3, 4.5],
               "min_child_weight": [1, 3, 5, 7],
               "n_estimators": [100, 250, 500, 1000]}

from sklearn.model_selection import RandomizedSearchCV
# Create RandomizedSearchCV Object
xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = "f1_micro",
                             cv = 10, verbose = 3, random_state = 40 )

# Fit the model
model_xgboost = xgb_rscv.fit(X_train, target_train)